In [3]:
import networkx as nx
from pymotif import Motif
import pandas as pd 
import datetime
import pytz
import os
import matplotlib
import matplotlib.pyplot as plt
import scipy
import jellyfish
import difflib
import networkx as nx
# import raphtoryclient 
from pymotif import Motif
import pyvis
from pyvis.network import Network
import matplotlib.pyplot as plt
import numpy as np
import string
import pydot
import seaborn as sns
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.metrics import PrecisionRecallDisplay

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import re

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model

import pickle

from sklearn.feature_extraction.text import TfidfVectorizer

import string
nltk.download('wordnet') # first-time use only
nltk.download('omw-1.4')
from tqdm import tqdm


import sys 
print(sys.version)

3.8.5 (v3.8.5:580fbb018f, Jul 20 2020, 12:11:27) 
[Clang 6.0 (clang-600.0.57)]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/HeloiseWright/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/HeloiseWright/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/HeloiseWright/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Preprocessing

In [4]:
data = pd.read_csv("1423.csv")
data.rename(columns={"Tweet Created At" : "created_at", "Retweeted User" : "in_reply_to_user_id", "Tweet By Handle" : "author_id", "Full Text" : "text", "Retweets Count" : "retweet_count", "Likes Count" : "like_count", "Source" : "source", "Tweet Hashtags" : "hashtags", "Tweet Mentions" : "user_mentions", "Tweet Symbols" : "symbols", "Reported Language" : "lang"}, inplace=True)
client_tweets = data[["author_id", "created_at", "in_reply_to_user_id", "lang", "retweet_count", "like_count", "source", "text", "hashtags", "user_mentions"]].copy()

def split_list(lst):
    if lst == "":
        return []
    else:
        return lst.split(";")
    
client_tweets["hashtags"] = client_tweets["hashtags"].astype("string")
client_tweets["hashtags"] = client_tweets["hashtags"].fillna("")
client_tweets["hashtags"] = client_tweets["hashtags"].apply(lambda x: split_list(x))

client_tweets["hashtags_count"]=client_tweets["hashtags"].apply(lambda x:len(x))

client_tweets["user_mentions"] = client_tweets["user_mentions"].astype("string")
client_tweets["user_mentions"] = client_tweets["user_mentions"].fillna("")
client_tweets["user_mentions"] = client_tweets["user_mentions"].apply(lambda x: split_list(x))
client_tweets["user_mentions_count"]=client_tweets["user_mentions"].apply(lambda x:len(x))

client_tweets = client_tweets[(client_tweets['lang']=='en')|(client_tweets['lang']=='und')]

client_tweets

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3135: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,author_id,created_at,in_reply_to_user_id,lang,retweet_count,like_count,source,text,hashtags,user_mentions,hashtags_count,user_mentions_count
0,MustafiMohamme1,2022-03-08 23:56:13+00:00,sanusi90064,en,0,0,Twitter for Android,"RT @sanusi90064: Iraq, Baghdad\n#iSupportRussi...","[IStandWithPutin, istandwithrussia, iSupportRu...",[sanusi90064],3,1
1,CrowYusuf,2022-03-08 23:55:57+00:00,NaN,und,1,2,Twitter Web App,@justsomeguycc #IStandWithRussia #IStandWithPutin,"[IStandWithPutin, IStandWithRussia]",[justsomeguycc],2,1
2,alendales,2022-03-08 23:53:51+00:00,5thSu,en,0,0,Twitter for Android,RT @5thSu: Good Afternoon Friends..😍🇷🇺💪❤️🌹\n#I...,"[Russia, Russian, IStandWithPutin]",[5thSu],3,1
3,Shandaren123,2022-03-08 23:53:49+00:00,Gpkumarreddy2,en,0,0,Twitter Web App,RT @Gpkumarreddy2: usa and nato hypocrites ...,"[IStandWithPutin, istandwithrussia]",[Gpkumarreddy2],2,1
5,ALICIA56483057,2022-03-08 23:52:42+00:00,worldobserver33,und,0,0,Twitter for Android,RT @worldobserver33: #IStandWithPutin #istandw...,"[RussianUkrainianWar, IStandWithPutin, istandw...",[worldobserver33],3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
235237,hbarjunky,2022-03-01 16:43:28+00:00,NaN,en,0,1,TweetDeck,@Mann_mit_Hund @de_rt_com Dr. Gideon Polya cou...,[IStandWithPutin],"[de_rt_com, Mann_mit_Hund]",1,2
235238,LordEnyo,2022-03-01 16:42:08+00:00,NaN,en,0,0,Twitter for Android,"If anyone here supports Putin, please, make th...","[FuckPutin, PutinWarCriminal, IStandWithPutin]",[],3,0
235240,Aman_SA_,2022-03-01 16:41:32+00:00,NaN,und,0,0,Twitter for Android,@KremlinRussia_E \n\n#IndiaStandWithPutin \n#I...,"[USA, Putin, UkraineRussiaWar, IStandWithPutin...",[KremlinRussia_E],6,1
235241,Aschrafie,2022-03-01 16:41:26+00:00,eleven45pm,en,0,0,Twitter Web App,RT @eleven45pm: Putin was on the ground in Afr...,[],[eleven45pm],0,1


In [5]:
data.rename(columns={"created_at" : "tweet_created_at", "User Created At" : "created_at", "User Description" : "description", "author_id" : "id", "Profile Country" : "location", "User Name" : "name", "Followers Count" : "followers_count", "Following Count" : "following_count", "User Listed Count" : "listed_count", "User URL" : "url", "User Handle" : "username", "Verified" : "verified"}, inplace=True)

In [6]:
client_users = data[["created_at", "description", "id", "location", "name", "followers_count", "following_count", "listed_count", "url", "username", "verified"]].copy()

In [7]:
tweet_no = client_users["id"].value_counts().to_frame()
tweet_no.reset_index(inplace=True)
tweet_no.columns = ["id","tweet_count"]

In [8]:
client_users.drop_duplicates(inplace=True)
client_users.reset_index(inplace=True)
client_users = pd.merge(client_users, tweet_no, on="id")
client_users

,index,created_at,description,id,location,name,followers_count,following_count,listed_count,url,username,verified,tweet_count
0,0,2021-08-28 19:10:25+00:00,አለማወቁን ያወቀ አዋቂ ነው,MustafiMohamme1,NaN,Mustafi MohammedAmin 🇪🇹🇪🇷🇩🇯🇸🇴🇰🇪,56,137,0,NaN,MustafiMohamme1,False,1
1,1,2020-11-10 21:14:01+00:00,🇮🇹 Pronouns: Screw SJWs/Screw Puritans\n\nAlte...,CrowYusuf,NaN,Yusuf-Pronouns: No Puritans/No SJWs #KenAkamatsu,59,126,1,NaN,CrowYusuf,False,8
2,2,2010-09-08 00:20:35+00:00,Из дворян...\nИз крестьян...\nИз рабочих...\nИ...,alendales,NaN,Александр Шаталов,1143,214,18,NaN,alendales,False,1
3,3,2022-03-07 00:06:01+00:00,"Just another russian bot... maybe, hard to tel...",Shandaren123,NaN,Shandaren,23,17,0,NaN,Shandaren123,False,8
4,4,2021-02-03 18:10:48+00:00,politics,Betty69319283,NaN,BettyTerefe,3150,1229,3,NaN,Betty69319283,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93480,235226,2021-02-18 08:38:41+00:00,Anti-social\nMechanical engineering (Trailers)...,DoumDoum19,NaN,Doum_Doum,72,45,0,NaN,DoumDoum19,False,1
93481,235227,2012-07-08 01:47:01+00:00,ذندگی کے دو ہی اصول ☝🏻الله اور ان کا۔ رسول ص...,XtasyRehman,United States,HRehman,196,459,0,NaN,XtasyRehman,False,1
93482,235228,2022-02-01 04:15:12+00:00,NaN,BinduHamal3,NaN,Bindu Hamal,1,9,0,NaN,BinduHamal3,False,1
93483,235235,2017-06-24 08:33:00+00:00,NaN,RomeoMakgaleme1,South Africa,Romeo Makgalemela,12,81,0,NaN,RomeoMakgaleme1,False,1


# Getting Sentiment

In [9]:
OUTPUT_SENTIMENT ="sentiment.csv"
OUTPUT_LABELS = "nlp_labelled.csv"
MAX_SEQUENCE_LENGTH = 50

In [10]:
model = load_model("~/BotDetection/sentiment_model.hdf5")
with open('~/BotDetection/sentiment_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

2022-08-02 14:29:56.195327: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "rt|rt@\S+|@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [12]:
def preprocess(text, stem=False):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    text = re.sub("istandwithputin", "i stand with putin", text)
    text = re.sub("istandwithrussia", "i stand with russia", text)
    text = re.sub("isupportrussia", "i support russia", text)
    text = re.sub("istandwithzelensky", "i stand with zelensky", text)
    text = re.sub("fuckputin", "fuck putin", text)
    text = re.sub("putinwarcriminal", "putin war criminal", text)
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [13]:
def decode_sentiment(score):
    if score>0.6:
        return "Positive"
    elif score<0.4:
        return "Negative"
    else:
        return "Neutral"

In [14]:
client_tweets.dropna(subset=["author_id", "text"], inplace=True)


In [15]:
puretext = client_tweets[["author_id", "text"]].copy()
puretext.dropna(inplace=True)
puretext

,author_id,text
0,MustafiMohamme1,"RT @sanusi90064: Iraq, Baghdad\n#iSupportRussi..."
1,CrowYusuf,@justsomeguycc #IStandWithRussia #IStandWithPutin
2,alendales,RT @5thSu: Good Afternoon Friends..😍🇷🇺💪❤️🌹\n#I...
3,Shandaren123,RT @Gpkumarreddy2: usa and nato hypocrites ...
5,ALICIA56483057,RT @worldobserver33: #IStandWithPutin #istandw...
...,...,...
235237,hbarjunky,@Mann_mit_Hund @de_rt_com Dr. Gideon Polya cou...
235238,LordEnyo,"If anyone here supports Putin, please, make th..."
235240,Aman_SA_,@KremlinRussia_E \n\n#IndiaStandWithPutin \n#I...
235241,Aschrafie,RT @eleven45pm: Putin was on the ground in Afr...


In [16]:
puretext.text = puretext.text.apply(lambda x: preprocess(x))

In [17]:
x = pad_sequences(tokenizer.texts_to_sequences(puretext.text), maxlen = MAX_SEQUENCE_LENGTH)

In [18]:
scores = model.predict(x, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]

23/23 [==============================] - 49s 2s/step


In [19]:
client_tweets["sentiment_score"] = scores
client_tweets["sentiment"] = y_pred_1d
client_tweets

,author_id,created_at,in_reply_to_user_id,lang,retweet_count,like_count,source,text,hashtags,user_mentions,hashtags_count,user_mentions_count,sentiment_score,sentiment
0,MustafiMohamme1,2022-03-08 23:56:13+00:00,sanusi90064,en,0,0,Twitter for Android,"RT @sanusi90064: Iraq, Baghdad\n#iSupportRussi...","[IStandWithPutin, istandwithrussia, iSupportRu...",[sanusi90064],3,1,0.581710,Neutral
1,CrowYusuf,2022-03-08 23:55:57+00:00,NaN,und,1,2,Twitter Web App,@justsomeguycc #IStandWithRussia #IStandWithPutin,"[IStandWithPutin, IStandWithRussia]",[justsomeguycc],2,1,0.834725,Positive
2,alendales,2022-03-08 23:53:51+00:00,5thSu,en,0,0,Twitter for Android,RT @5thSu: Good Afternoon Friends..😍🇷🇺💪❤️🌹\n#I...,"[Russia, Russian, IStandWithPutin]",[5thSu],3,1,0.947319,Positive
3,Shandaren123,2022-03-08 23:53:49+00:00,Gpkumarreddy2,en,0,0,Twitter Web App,RT @Gpkumarreddy2: usa and nato hypocrites ...,"[IStandWithPutin, istandwithrussia]",[Gpkumarreddy2],2,1,0.554189,Neutral
5,ALICIA56483057,2022-03-08 23:52:42+00:00,worldobserver33,und,0,0,Twitter for Android,RT @worldobserver33: #IStandWithPutin #istandw...,"[RussianUkrainianWar, IStandWithPutin, istandw...",[worldobserver33],3,1,0.799398,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235237,hbarjunky,2022-03-01 16:43:28+00:00,NaN,en,0,1,TweetDeck,@Mann_mit_Hund @de_rt_com Dr. Gideon Polya cou...,[IStandWithPutin],"[de_rt_com, Mann_mit_Hund]",1,2,0.468949,Neutral
235238,LordEnyo,2022-03-01 16:42:08+00:00,NaN,en,0,0,Twitter for Android,"If anyone here supports Putin, please, make th...","[FuckPutin, PutinWarCriminal, IStandWithPutin]",[],3,0,0.555652,Neutral
235240,Aman_SA_,2022-03-01 16:41:32+00:00,NaN,und,0,0,Twitter for Android,@KremlinRussia_E \n\n#IndiaStandWithPutin \n#I...,"[USA, Putin, UkraineRussiaWar, IStandWithPutin...",[KremlinRussia_E],6,1,0.735688,Positive
235241,Aschrafie,2022-03-01 16:41:26+00:00,eleven45pm,en,0,0,Twitter Web App,RT @eleven45pm: Putin was on the ground in Afr...,[],[eleven45pm],0,1,0.593956,Neutral


# Raphtory

In [20]:
# #Sanitising input for Raphtory
# raphtory_tweets = client_tweets.drop('text',axis=1)
# raphtory_tweets = raphtory_tweets.drop('hashtags',axis=1)
# raphtory_tweets = raphtory_tweets.drop('user_mentions',axis=1)
# raphtory_tweets.to_csv(OUTPUT_SENTIMENT, index=False)

In [21]:
# from pathlib import Path
# from pyraphtory.context import PyRaphtory
# from pyraphtory.vertex import Vertex
# from pyraphtory.steps import Iterate, Step

# import os
# import subprocess
# import sys

# os.environ['defaultLogLevel'] = "DEBUG"

# pr = PyRaphtory(spout_input=Path('sentiment.csv'), builder_script=Path('builder.py'),  builder_class='BotsGraphBuilder', mode='batch', logging=True).open()
# rg = pr.graph()


ModuleNotFoundError: No module named 'pyraphtory'

In [ ]:
# local_sink = pr.local_sink()

# cols = ["inDegree", "outDegree", "degree","triangleCount","prlabel","cclabel","twoHopPaths"]

# tracker = rg.at(1646697600)\
#             .past()\
#             .transform(pr.page_rank())\
#             .transform(pr.connected_components())\
#             .transform(pr.degree())\
#             .transform(pr.two_hops_path(set([])))\
#             .transform(pr.local_triangle_count())\
#             .select(cols) \
#             .write_to(local_sink)\
#             .wait_for_job()


# raphtory_results = pr.results_to_dataframe(local_sink.results(),cols)

In [ ]:
# raphtory_results

# Length of Time Between Tweeting

In [22]:
from datetime import datetime
client_tweets['created_at']=client_tweets['created_at'].apply(lambda x: datetime.fromisoformat(x).timestamp())
dfTime=client_tweets.sort_values(['author_id','created_at'],ascending=True).groupby('author_id').head(200)
dfTime['diff'] =   dfTime["created_at"] - dfTime['created_at'].shift()
dfTime['match'] = dfTime.author_id.eq(dfTime.author_id.shift())
dfTime.loc[dfTime['match'] ==False , 'diff'] = None 

dfTime['diff'] = dfTime['diff'].fillna(0)
dfTime= dfTime.groupby(['author_id'])['diff'].mean().to_frame()
dfTime.reset_index(inplace=True)

dfTime

,author_id,diff
0,000000o0oo0,66.500000
1,00000d0,7261.230769
2,0007vijay786,0.000000
3,000Nancy000,0.000000
4,000devdarbar000,0.000000
...,...,...
88180,zzmkhonza,0.000000
88181,zzurrahman90,0.000000
88182,zzw0571,0.000000
88183,zzxxu1,2870.300000


# Getting Tweet Similarity and Lexical Diversity

In [23]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [24]:
nltk.download('punkt')
train_text = pd.read_csv("~/BotDetection/train_text.csv")
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
TfidfVec.fit(train_text)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/HeloiseWright/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


TfidfVectorizer(stop_words='english',
                tokenizer=<function LemNormalize at 0x169aec820>)

In [25]:
def cos_similarity(textlist):
    if textlist == []:
        return 0
    else:
        tfidf = TfidfVec.transform(textlist)
        return (tfidf * tfidf.T).toarray()

In [26]:
def token_type_ratio(string):
    string = re.sub(r'[^\w]', ' ', string)
    string = string.lower()
    tokens = nltk.word_tokenize(string)
    types = nltk.Counter(tokens)
    if tokens == []:
        return 0
    else:
        return (len(types)/len(tokens))*100

In [27]:
def average(array):
    average = []
    for i in range(len(array)):
        for j in range(len(array)):
            if i != j:
                average.append(j)
    if len(average) == 0:
        return 0
    else:
        return sum(average)/len(average)

In [28]:
users = puretext["author_id"].unique().tolist()

In [29]:
from os.path import exists
import pickle

sim_dict = {}

if(exists('sim_dict.pickle')):
    with open('sim_dict.pickle', 'rb') as handle: #for reading
        sim_dict = pickle.load(handle) 
else:
    count=0
    for a_id in tqdm(users):
        if(count <=126000):
            tweets = puretext[puretext["author_id"]==a_id].text.tolist()
            sim = cos_similarity(tweets)
            sim_dict[a_id] = average(sim)
            count+=1
            
    with open('sim_dict.pickle', 'wb') as handle:
        pickle.dump(sim_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)            
        
df_sim = pd.DataFrame(sim_dict.items(), columns=["author_id", "tweet_similarity"])

100%|█████████████████████████████████████| 88185/88185 [36:57<00:00, 39.78it/s]


In [30]:
ttr_dict = {}

if(exists('ttr_dict.pickle')):
    with open('ttr_dict.pickle', 'rb') as handle: #for reading
        ttr_dict = pickle.load(handle)    
else:    
    count=0
    for a_id in tqdm(users):
        if(count <=126000):
            count+=1
            tweets = puretext[puretext["author_id"]==a_id].text.tolist()
            ttrs = [token_type_ratio(tweet) for tweet in tweets]
            if ttrs == []:
                ttr_dic[a_id] = 0
            else:
                ttr_dict[a_id] = sum(ttrs)/len(ttrs)
    with open('ttr_dict.pickle', 'wb') as handle:
        pickle.dump(ttr_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)                  
            
df_ttr = pd.DataFrame(ttr_dict.items(), columns=["author_id", "lexical_diversity"])

100%|█████████████████████████████████████| 88185/88185 [35:26<00:00, 41.47it/s]


In [31]:
df_sent = client_tweets.groupby("author_id")["sentiment_score"].mean().to_frame()
df_sent.reset_index(inplace=True)
df_sent.columns = ["author_id","sentiment"]

In [32]:
new_data = pd.merge(df_sim, df_ttr, on="author_id")
new_data = pd.merge(new_data, df_sent, on="author_id")
new_data.to_csv(OUTPUT_LABELS, index=False)

# Profile Included Data

### Follower Following Ratio

In [33]:
dfFF = client_users.copy()
dfFF=dfFF[pd.to_numeric(dfFF['following_count'], errors='coerce').notnull()]
dfFF=dfFF[pd.to_numeric(dfFF['following_count'], errors='coerce').notnull()]


dfFF['followers_count'] = dfFF['followers_count'].astype('Int64')
dfFF['following_count'] = dfFF['following_count'].astype('Int64')

# dfUsers['followers_count'] = dfUsers['followers_count'].fillna(0)
# dfUsers['following_count'] = dfUsers['following_count'].fillna(0)
# dfUsers['followers_count']=dfUsers[pd.to_numeric(dfUsers['followers_count'], errors='coerce').notnull()]
# dfUsers['following_count']=dfUsers[pd.to_numeric(dfUsers['following_count'], errors='coerce').notnull()]


client_users['f/f']=dfFF['followers_count']/dfFF['following_count']


### Listed Count / Tweet Count

In [34]:
client_users['l/t']= client_users['listed_count'].astype('int64')/client_users['tweet_count'].astype('int64')


### UserName / Name Similarity

In [35]:
client_users['name'] = client_users['name'].str.replace(" ","")
client_users["name"] = client_users["name"].str.lower()
client_users["username"] = client_users["username"].str.lower()

client_users['n/u']= client_users.apply(lambda x: difflib.SequenceMatcher(None, str(x['name']), str(x['username'])).ratio(), axis=1)


### Tweets And Account age


In [36]:
dfProfileTA = client_users.copy()

In [37]:
dfProfileTA.sort_values('tweet_count')
my_date = 1658244252
dfProfileTA['created_at']=dfProfileTA['created_at'].astype('datetime64')
dfProfileTA['DateDiff']=dfProfileTA['created_at'].apply(lambda x:x.tz_localize(None))
dfProfileTA['DateDiff']=dfProfileTA['DateDiff'].apply(lambda x:x.timestamp())
dfProfileTA['DateDiff']=(my_date-dfProfileTA['DateDiff'])/dfProfileTA['tweet_count']



# We're all in this together

In [40]:
#### MERGE EVERYTHING HERE
#client_users #Original tweet data
#new_data #Sementic information
#dfFF #Followers and Following
#dfProfileTA #Number of Tweets over account age
#dfTime #Time between tweets
#raphtory_results 

hashtag= client_tweets.groupby(['author_id'])['hashtags_count'].mean().to_frame()
usr= client_tweets.groupby(['author_id'])['user_mentions_count'].mean().to_frame()
ret= client_tweets.groupby(['author_id'])['retweet_count'].mean().to_frame()
lik= client_tweets.groupby(['author_id'])['like_count'].mean().to_frame()

FullData= pd.merge(client_users, new_data, how = 'inner', right_on=['author_id'],left_on=['id'])
FullData = FullData.merge(dfTime,how = 'inner', right_on=['author_id'],left_on=['id'])
FullData = FullData.merge(dfFF,how = 'inner', right_on=['id'],left_on=['id'])
FullData = FullData.merge(dfProfileTA,how = 'inner', right_on=['id'],left_on=['id'])
# FullData = FullData.merge(raphtory_results,how = 'inner', right_on=['name'],left_on=['id'])
FullData = FullData.merge(hashtag,how = 'inner', right_on=['author_id'],left_on=['id'])
FullData = FullData.merge(usr,how = 'inner', right_on=['author_id'],left_on=['id'])
FullData = FullData.merge(ret,how = 'inner', right_on=['author_id'],left_on=['id'])
FullData = FullData.merge(lik,how = 'inner', right_on=['author_id'],left_on=['id'])
FullData = FullData[['id','cclabel','f/f','l/t','n/u','DateDiff','verified','diff','sentiment','tweet_similarity','lexical_diversity','hashtags_count','user_mentions_count','retweet_count','like_count']]
# FullData = FullData[['id','triangleCount','cclabel','f/f','l/t','n/u','DateDiff','prlabel','verified','diff','inDegree','outDegree','degree','sentiment','tweet_similarity','lexical_diversity','hashtags_count','user_mentions_count','retweet_count','like_count']]

KeyError: "['l/t', 'cclabel', 'f/f', 'n/u'] not in index"

In [ ]:
FullData

# Model Predictions


In [ ]:
with open("bot_prediction_model.pkl", "rb") as f:
    rf = pickle.load(f)

In [ ]:
no_id = FullData.drop('id',axis=1)
no_id.replace([np.nan,np.inf,-np.inf,None,'NaN'], 0,inplace=True)
no_id = no_id[~no_id['f/f'].isin([np.nan,np.inf,-np.inf,None])]


predictions = pd.DataFrame(rf.predict(X=no_id.values))

FullData=FullData.join(predictions)
FullData.rename(columns={0 : "label"}, inplace=True)
FullData

In [ ]:
df_for_graph=client_tweets.merge(FullData,how='inner',right_on='id',left_on='author_id')
df_for_graph["in_reply_to_user_id"] = df_for_graph["in_reply_to_user_id"].fillna("0")
#df_for_graph=df_for_graph.sort_values(['created_at'],ascending=True)
df_for_graph
df_for_graph.to_csv("final_data.csv")

In [ ]:
# pyvis_nt.inherit_edge_colors(True)
# test = pd.read_csv('/Users/pometry/Desktop/sentiments1.csv')
import math
pyvis_nt = Network('900px', '1400px', directed = True)
for index, row in df_for_graph.iterrows():
    if row["label"] == "bot":
        pyvis_nt.add_node(row["author_id"],  shape='image', image ="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSGZM6dmn0zwPv0E6CYfGve08tCFQvRHt9j9g&usqp=CAU")
    if row["label"] == "human":
        pyvis_nt.add_node(row["author_id"], shape='image', image ="https://i0.wp.com/caravetclinic.com/wp-content/uploads/2016/07/person-icon-blue.png?ssl=1")

    if row["in_reply_to_user_id"] != "0":    
        pyvis_nt.add_node(row["in_reply_to_user_id"], shape='image', image ="https://i0.wp.com/caravetclinic.com/wp-content/uploads/2016/07/person-icon-blue.png?ssl=1")
        
        
        if row["sentiment_x"] == "Positive":
            pyvis_nt.add_edge(row["author_id"], row["in_reply_to_user_id"], color='green', width=4)
        elif row["sentiment_x"] == "Neutral":
            pyvis_nt.add_edge(row["author_id"], row["in_reply_to_user_id"], color='black', width=1)
        else: 
            pyvis_nt.add_edge(row["author_id"], row["in_reply_to_user_id"], color='red', width=6)


        

    
    
pyvis_nt.toggle_physics(True)
pyvis_nt.set_edge_smooth('smooth')
pyvis_nt.show_buttons()
pyvis_nt.show('nx.html')